## Importaciones

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

## Cargar datos

He plasmado todos los datos dentro de archivos CSV, que están en la carpeta ["data"](data/), por lo que ahora para cargarlos sólo hay que crear un DataFrame de Pandas vacío y juntar en él los datos de todos los archivos. Un bucle "for" nos hace el trabajo independientemente de que agregue más archivos en un futuro

In [6]:
dataset = pd.DataFrame()

for file in os.listdir("./data"):
    dataset = pd.concat([dataset, pd.read_csv(os.path.join("./data", file), delimiter=",", quotechar='"', header=None)])

Ya que es interesante que las respuestas tengan elementos escritos en MarkDown, pero no se pueden guardar en CSV, ésta es la solución:
- Creamos una "sintaxis" nueva en la que representamos los elementos con otros caracteres que sí podamos tener en CSV
- Definimos esa sintaxis en un diccionario dentro del código, guardando cada representación junto con su elemento
- Usamos un bucle for para reemplazar todo en el DataFrame, ya que en el CSV no podíamos tenerlos pero en el DataFrame sí

In [9]:
markdown = {
    "&001": "\n"
}

In [13]:
for a, n in markdown.items():
    dataset[1] = dataset[1].str.replace(a, n)

In [12]:
print(dataset)

                    ¿Qué es una reliquia?  \
0  ¿Qué puedes conseguir de una reliquia?   
0                                     NaN   

  Si has jugado otros juegos, por suerte o por desgracia sabrás que hay unas cajas que compras por dinero real y te pueden tocar elementos aleatorios del juego. Bien, Warframe tiene un elemento similar pero que no es de pago; las reliquias  \
0  Lo más "exclusivo" que pueden darte son piezas...                                                                                                                                                                                              
0                                                NaN                                                                                                                                                                                              

     ¿Qué modelo eres?  \
0                  NaN   
0  ¿Quién te ha hecho?   

  Soy Cephalon X, el céfalon más inteligente hasta la f

## Preparación

Uno no prepara los datos y comienza el entrenamiento ya, hay que hacer un par de cosas primero. Entre ellas, configurar un checkpointer

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint(
    filepath="./checkpoints/model.tf"
    monitor="val_loss"
    mode=min
    save_best_only=True
)

Éste objeto de Keras se encarga de ir guardando el modelo en mitad del entrenamiento. Al no tener ninguna variable el nombre (aunque se le podrían poner), en la ruta sólo va a haber un modelo siempre, y gracias al resto de parámetros, siempre será el que haya sacado la pérdida de validación (val_loss) más baja